Metrics and Stats: Finally, visibility
------------------------------------------------------

Riemann, InfluxDB, Grafana, and more

Previous visibility
----------------------
* Check Marathon / Mesos directly
* Kibana
* More recently, Bigboss stats

![zener](zener.png)


Metrics stack:
-------------------------

* Riemann - Aggregator (like Logstash)
* InfluxDB - Time series database (like Elastic Search)
* Grafana - Dashboard (like Kibana)
* Flapjack - Alerting
* Sensu - Like Nagios but newer

Riemann
--------
* Written in Clojure (Lisp on the JVM)
* All configuration also done in Clojure
* Powerful stream processor
* Exposes a rich API for interacting with data streams, plus Clojure's builtin tooling

Basic Clojure
---------------
* Lisp, but compiles to Java bytecode (and runs on the JVM)
* Dynamic typing (like most Lisps)
* Basically everything is immutable

In [1]:
; Some basics
; Statements are called "forms" (basically, a block of code)

"this is a string"

"this is a string"

In [2]:
; Numbers

5

5

In [3]:
; Function calls

(+ 1 2)

3

In [4]:
; And definitions

(def foo 5)

#'clojure.core/foo

In [5]:
(defn add_five
  [input]
  (+ input 5))

#'clojure.core/add_five

In [6]:
; Control flow

(if true
  "This is true"
  "Nevermind its false")

"This is true"

In [7]:
(let [tmp (+ 2 3)]
  (+ 10 tmp))

15

In [8]:
(when true
  (println "Still true")
  (println "but i can have multiple forms here"))

Still true
but i can have multiple forms here


nil

In [9]:
; falsiness works like Lua (only nil and false are falsey)

(do
  (println (if 0 "0 is not falsey" "0 is falesy"))
  (println (if nil "nil is not falsey" "nil is falsey")))

0 is not falsey
nil is falsey


nil

Data types and structures
-----------------

In [10]:
; Basics like strings and numbers, nil, true/false (note: 
; all strings are double quoted)
; Keywords, often used as keys in Maps, which also serve 
; as a "get" function

(:foo {:foo "1" :bar "2"})

"1"

In [11]:
; Maps (remember, its all lists, so you just alternate key/value, 
; no : or = or ,)

{:a "a" :b "123" "qwer" 25}

{:a "a", :b "123", "qwer" 25}

In [12]:
; Vectors (basically, arrays); good for adding things to the end, and 
; can "get" values by index efficiently

(do 
  (println [1 2 "asdf" true])
  (println (get [5 6 7] 1))
  (println (conj [1 2 3] 4)))

[1 2 asdf true]
6
[1 2 3 4]


nil

In [13]:
; Lists, similar to vectors, but items are added to the front; "nth" 
; to get back elements is also slower

(do
  (println '(1 2 "asdf" true))
  (println (nth '(5 6 7) 1))
  (println (conj '(1 2 3) 4)))

(1 2 asdf true)
6
(4 1 2 3)


nil

In [14]:
; Sets (this is a hash set, specifically); just a unique collection 
; of values

(do 
  (println #{1 2 "asdf" true})
  (println (hash-set 1 1 3 4 "qwer" false false)))

#{1 2 true asdf}
#{1 3 4 qwer false}


nil

Other Important Things
------------------

In [15]:
; Remember everything is immutable, and `def` is assignment, 
; so `=` is just an eq comparison

(= 1 2)

false

In [16]:
; Functions return the value of the last thing they do

(do 
  (defn foo
    []
    (+ 1 2)
    "this is returned")

  (foo))

"this is returned"

In [17]:
; Functions can have multiple arities (number of arguments)

(do
  (defn baz
    ([x]
      (println (str "you entered: " x)))
    ([]
      (baz "default value")))
  (baz)
  (baz "something else"))

you entered: default value
you entered: something else


nil

Riemann Basics
----------------
* Config files are just regular clojure; you can define variables, functions, whatever
* All the work comes from the `streams` function Riemann provides: all your stream processing happens in there

In [ ]:
; The `where` function basically filters your data; here, 
; we use the provided `service` function

(streams
  (where (service "my-service-name")
    send-influx))

In [ ]:
; You can also "group-by" data, with `by`

(streams
  (by :service
    send-influx))

In [ ]:
; The `rate` function rolls up data at an interval

(streams
  (rate 1
    (send-influx)))

In [ ]:
; The `with` function lets you pass along an attr on a stream

(streams
  (where (service "foobar")
    (with :service "renamed-foobar"
      send-influx)))

In [ ]:
; There's also a super helpful `percentiles` function, to let
; you see how time-based metrics really look (not just the avg)

(streams
  (where (service "api-request-time")
    (percentiles 1 [0.5 0.95 0.99]
      send-influx)))